In [56]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses

In [2]:
print(tf.__version__)

2.18.0


In [3]:
pip install spacy

   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
   ---------- ----------------------------- 3.1/11.8 MB 16.8 MB/s eta 0:00:01
   ------------------------ --------------- 7.3/11.8 MB 18.1 MB/s eta 0:00:01
   ---------------------------------------  11.5/11.8 MB 19.0 MB/s eta 0:00:01
   ---------------------------------------- 11.8/11.8 MB 17.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 13.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/632.6 kB ? eta -:--:--
   ---------------------------------------- 632.6/632.6 kB 7.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 15.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   -------------------------- ------------- 4.2/6.3 MB 19.4 MB/s eta 0:00:01
   ------------------------


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!python -m spacy download 

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------- -------------------------------- 2.4/12.8 MB 14.9 MB/s eta 0:00:01
     ------------------- -------------------- 6.3/12.8 MB 16.1 MB/s eta 0:00:01
     ------------------------------- ------- 10.2/12.8 MB 17.2 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 16.0 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [57]:
import spacy
nlp=spacy.load('en_core_web_sm')

In [58]:
about_text=('I am a python developer currently'
        'working for a London-based Fintech'
        'company.I am interested in learning'
            'Natural Language Processing.')
about_doc=nlp(about_text)
sentences=list(about_doc.sents)
len(sentences)
for sentence in sentences:
    print(sentence)

I am a python developer currentlyworking for a London-based Fintechcompany.
I am interested in learningNatural Language Processing.


In [59]:
for token in about_doc:
    print(token,token.idx)

I 0
am 2
a 5
python 7
developer 14
currentlyworking 24
for 41
a 45
London 47
- 53
based 54
Fintechcompany 60
. 74
I 75
am 77
interested 80
in 91
learningNatural 94
Language 110
Processing 119
. 129


In [61]:
from spacy.tokenizer import Tokenizer
custom_nlp=spacy.load('en_core_web_sm')
prefix_re=spacy.util.compile_prefix_regex(custom_nlp.Defaults.prefixes)
suffix_re=spacy.util.compile_suffix_regex(custom_nlp.Defaults.suffixes)
infix_re=re.compile(r'''[-~]''')
def customize_tokenizer(nlp):
    return Tokenizer(nlp.vocab,prefix_search=prefix_re.search,
                     suffix_search=suffix_re.search,
                     infix_finditer=infix_re.finditer,
                     token_match=None
                    )
custom_nlp.tokenizer=customize_tokenizer(custom_nlp)
cutom_tokenizer_about_doc=custom_nlp(about_text)
print([token.text for token in cutom_tokenizer_about_doc])

['I', 'am', 'a', 'python', 'developer', 'currentlyworking', 'for', 'a', 'London', '-', 'based', 'Fintechcompany.I', 'am', 'interested', 'in', 'learningNatural', 'Language', 'Processing', '.']


In [62]:
for token in about_doc:
    if not token.is_stop:
        print(token)

python
developer
currentlyworking
London
-
based
Fintechcompany
.
interested
learningNatural
Language
Processing
.


In [63]:
con_help_txt=('Raj is helping organize a developer'

     'conference on Applications of Natural Language'

     ' Processing. He keeps organizing local Python meetups'

     ' and several internal talks at his workplace.')
con_help_doc=nlp(con_help_txt)
for token in con_help_doc:
    print(token,token.lemma_)

Raj raj
is be
helping helping
organize organize
a a
developerconference developerconference
on on
Applications Applications
of of
Natural Natural
Language Language
Processing processing
. .
He he
keeps keep
organizing organize
local local
Python Python
meetups meetup
and and
several several
internal internal
talks talk
at at
his his
workplace workplace
. .


In [64]:
from collections import Counter
complete_txt=('J K is a Python developer currently'

    'working for a London-based Fintech company. He is'

    ' interested in learning Natural Language Processing.'

    ' There is a conference happening on 21 June'

     ' 2019 in London. It is titled "Applications of Natural'

    ' Language Processing". There is a helpline number '

    ' available . J is helping organize it.'

    ' He keeps organizing local Python meetups and several'

    ' internal talks at his workplace. J is also presenting'

     ' a talk. The talk will introduce the reader about "Use'

     ' cases of Natural Language Processing in Fintech".'

     ' Apart from his work, he is very passionate about music.'

     ' J is learning to play the Piano. He has enrolled '

     ' himself in the weekend batch of Great Piano Academy.'

     ' Great Piano Academy is situated in Mayfair or the City'


     ' of London and has world-class piano instructors.')

complete_doc=nlp(complete_txt)
words=[token.text for token in complete_doc
       if not token.is_stop and not token.is_punct]
word_freq=Counter(words)
common_words=word_freq.most_common(5)
print(common_words)

[('J', 4), ('London', 3), ('Natural', 3), ('Language', 3), ('Processing', 3)]


In [65]:
unique_words=[word for(word,freq)in word_freq.items()if freq==1]
print(unique_words)

['K', 'developer', 'currentlyworking', 'based', 'company', 'interested', 'conference', 'happening', '21', 'June', '2019', 'titled', 'Applications', 'helpline', 'number', 'available', 'helping', 'organize', 'keeps', 'organizing', 'local', 'meetups', 'internal', 'talks', 'workplace', 'presenting', 'introduce', 'reader', 'Use', 'cases', 'Apart', 'work', 'passionate', 'music', 'play', 'enrolled', 'weekend', 'batch', 'situated', 'Mayfair', 'City', 'world', 'class', 'piano', 'instructors']


In [66]:
piano_txt=('Piano Academy is situated'
           'in Mayfaire or the city of London and has'
           'world-class piano instructors.')
piano_doc=nlp(piano_txt)
for ent in piano_doc.ents:
    print(ent.text,ent.start_char,ent.end_char,
          ent.label_,spacy.explain(ent.label_))

Piano Academy 0 13 ORG Companies, agencies, institutions, etc.
Mayfaire 28 36 ORG Companies, agencies, institutions, etc.
London 52 58 GPE Countries, cities, states


In [67]:
# Bag of Words Model

In [68]:
doc_a='this document is first document'
doc_b='this document is the second document'

In [69]:
bag_of_words_a=doc_a.split(' ')
bag_of_words_b=doc_b.split(' ')

In [70]:
unique_words_set=set(bag_of_words_a).union(set(bag_of_words_b))
print(unique_words_set)

{'is', 'first', 'document', 'second', 'this', 'the'}


In [71]:
dict_a=dict.fromkeys(unique_words_set,0)
print(dict_a)

{'is': 0, 'first': 0, 'document': 0, 'second': 0, 'this': 0, 'the': 0}


In [72]:
for word in bag_of_words_a:
    dict_a[word]+=1
print(dict_a)

{'is': 1, 'first': 1, 'document': 2, 'second': 0, 'this': 1, 'the': 0}


In [73]:
dict_b=dict.fromkeys(unique_words_set,0)

In [74]:
for word in bag_of_words_b:
    dict_b[word]+=1
print(dict_b)

{'is': 1, 'first': 0, 'document': 2, 'second': 1, 'this': 1, 'the': 1}


In [75]:
def compute_term_frequency(word_dictionary,bag_of_words):
    term_frequency_dictionary={}
    length_of_bag_of_words=len(bag_of_words)
    for word,count in word_dictionary.items():
        term_frequency_dictionary[word]=count/float(length_of_bag_of_words)
        return term_frequency_dictionary
print(compute_term_frequency(dict_a,bag_of_words_a))

{'is': 0.2}


In [76]:
import math
def compute_inverse_document_frequency(full_doc_list):
    idf_dic={}
    length_of_doc_list=len(full_doc_list)

In [77]:
idf_dict=dict.fromkeys(full_doc_list[0].keys(),0)
for word,value in idf_dict.items():
    idf_dict[word]=math.log(length_of_doc_list/(float(value)+1))
return idf_dict
final_idf_dict=compute_inverse_document_frequency([dict_a,dict_b])
print(final_idf_dict)

NameError: name 'full_doc_list' is not defined

In [78]:
import math

def compute_inverse_document_frequency(full_doc_list):
    length_of_doc_list = len(full_doc_list)  # Number of documents

    # ✅ Initialize IDF dictionary with words from the first document
    idf_dict = dict.fromkeys(full_doc_list[0].keys(), 0)

    # ✅ Count how many documents contain each word
    for doc in full_doc_list:
        for word, value in doc.items():
            if value > 0:  # The word appears in the document
                idf_dict[word] += 1

    # ✅ Compute IDF using log(N / (df + 1))
    for word in idf_dict:
        idf_dict[word] = math.log(length_of_doc_list / (idf_dict[word] + 1))

    return idf_dict  # ✅ Correct indentation

# Example dictionaries (word counts per document)
dict_a = {'this': 1, 'document': 2, 'is': 1, 'first': 1, 'the': 0, 'second': 0}
dict_b = {'this': 1, 'document': 2, 'is': 1, 'first': 0, 'the': 1, 'second': 1}

# ✅ Pass the correctly defined full_doc_list
full_doc_list = [dict_a, dict_b]

final_idf_dict = compute_inverse_document_frequency(full_doc_list)
print(final_idf_dict)


{'this': -0.40546510810816444, 'document': -0.40546510810816444, 'is': -0.40546510810816444, 'first': 0.0, 'the': 0.0, 'second': 0.0}
